# Imports & Load Data
作業に必要なライブラリをインポートして、 以下のデータを読み込みます。

* stock_price : 株価情報
* stock_list : 銘柄情報
* stock_fin : 財務諸表
* stock_labels : 目的変数

In [ ]:
# shap用にg++とgccをインストールします
! apt-get update
! apt-get install -y --no-install-recommends g++ gcc

# 必要なライブラリをインストールします
! pip install shap==0.37.0 slicer==0.0.3 xgboost==1.3.0.post0

In [1]:
!pip install tensorflow

    100% |████████████████████████████████| 394.3MB 24kB/s  eta 0:00:0111 day, 23:59:03                     | 6.5MB 12.9MB/s eta 0:00:31    5% |█▉                              | 22.3MB 7.9MB/s eta 0:00:48    5% |█▉                              | 22.9MB 9.0MB/s eta 0:00:42    20% |██████▋                         | 81.5MB 13.4MB/s eta 0:00:24    23% |███████▋                        | 93.4MB 16.4MB/s eta 0:00:19�██████▌                    | 141.3MB 14.2MB/s eta 0:00:18    41% |█████████████▏                  | 162.2MB 11.6MB/s eta 0:00:21    45% |██████████████▌                 | 178.3MB 6.7MB/s eta 0:00:33    50% |████████████████                | 198.3MB 16.5MB/s eta 0:00:12    52% |████████████████▉               | 207.7MB 16.9MB/s eta 0:00:12 |█████████████████▌              | 215.4MB 17.7MB/s eta 0:00:11    55% |█████████████████▉              | 219.7MB 10.4MB/s eta 0:00:17    57% |██████████████████▎             | 225.9MB 39.6MB/s eta 0:00:05    58% |██████████████████▋             

    100% |████████████████████████████████| 788kB 9.3MB/s eta 0:00:01    68% |█████████████████████▉          | 532kB 19.7MB/s eta 0:00:01
    100% |████████████████████████████████| 788kB 6.1MB/s eta 0:00:01B 3.7MB/s eta 0:00:01
    100% |████████████████████████████████| 102kB 3.7MB/s a 0:00:01
    100% |████████████████████████████████| 163kB 12.2MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 3.9MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 3.7MB/s ta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/b1/c2/ed/d62208260edbd3fa7156545c00ef966f45f2063d0a84f8208a
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built wrapt termcolor
tables 3.5.1 requires mock>=2.0, which is not installed.
astroid 2.2.5 requires typed-ast>=1.3.0; implementation_name == "cpython", which is not installed.
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully 

In [2]:
!pip install keras
!pip install tensorflow

In [3]:
!pip install lightgbm

    100% |████████████████████████████████| 2.0MB 1.8MB/s ta 0:00:011


In [4]:
!pip install catboost

    100% |████████████████████████████████| 67.3MB 283kB/s ta 0:00:011    29% |█████████▍                      | 19.7MB 8.4MB/s eta 0:00:06    31% |██████████                      | 21.2MB 18.1MB/s eta 0:00:03    32% |██████████▍                     | 21.9MB 14.2MB/s eta 0:00:04    36% |███████████▊                    | 24.6MB 16.1MB/s eta 0:00:03�█               | 35.9MB 24.5MB/s eta 0:00:02    54% |█████████████████▍              | 36.6MB 7.7MB/s eta 0:00:04    78% |█████████████████████████       | 52.7MB 5.7MB/s eta 0:00:03
    100% |████████████████████████████████| 13.2MB 3.0MB/s eta 0:00:01   42% |█████████████▌                  | 5.6MB 2.4MB/s eta 0:00:04
  Stored in directory: /root/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [5]:
import os
import pickle
import sys
import warnings
from glob import glob

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import xgboost
from scipy.stats import spearmanr
from sklearn.ensemble import (
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm.auto import tqdm


# 表示用の設定を変更します
%matplotlib inline
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.width = 120

In [6]:
# python 3.7.3であることを確認します
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [7]:
# データセット保存先ディレクトリ（""の中身はご自身の環境に合わせて定義してください。）
dataset_dir="/path/to"

In [8]:
# 読み込むファイルを定義します。
inputs = {
    "stock_list": f"{dataset_dir}/stock_list.csv.gz",
    "stock_price": f"{dataset_dir}/stock_price.csv.gz",
    "stock_fin": f"{dataset_dir}/stock_fin.csv.gz",
    # 本チュートリアルでは使用しないため、コメントアウトしています。
    # "stock_fin_price": f"{dataset_dir}/stock_fin_price.csv.gz",
    "stock_labels": f"{dataset_dir}/stock_labels.csv.gz",
}

# ファイルを読み込みます
dfs = {}
for k, v in inputs.items():
    print(k)
    dfs[k] = pd.read_csv(v)

stock_list
stock_price
stock_fin
stock_labels


In [8]:
COLUMNS = ['Result_FinancialStatement FiscalYear', 'Result_FinancialStatement NetSales',
       'Result_FinancialStatement OperatingIncome', 'Result_FinancialStatement OrdinaryIncome',
       'Result_FinancialStatement NetIncome', 'Result_FinancialStatement TotalAssets',
       'Result_FinancialStatement NetAssets', 'Result_FinancialStatement CashFlowsFromOperatingActivities',
       'Result_FinancialStatement CashFlowsFromFinancingActivities',
       'Result_FinancialStatement CashFlowsFromInvestingActivities', 'Forecast_FinancialStatement FiscalYear',
       'Forecast_FinancialStatement NetSales', 'Forecast_FinancialStatement OperatingIncome',
       'Forecast_FinancialStatement OrdinaryIncome', 'Forecast_FinancialStatement NetIncome',
       'Result_Dividend FiscalYear', 'Result_Dividend QuarterlyDividendPerShare',
       'Result_Dividend AnnualDividendPerShare', 'Forecast_Dividend FiscalYear',
       'Forecast_Dividend QuarterlyDividendPerShare', 'Forecast_Dividend AnnualDividendPerShare',
       'IssuedShareEquityQuote IssuedShare','Section/Products', '33 Sector(Code)', '17 Sector(Code)']

In [9]:
FEATURES = ['MA_gap_2month',
            'MA_gap_3month',
            'volatility_2month',
            'volatility_3month',
            'Result_Dividend FiscalYear',
            'return_3month',
            'Forecast_Dividend FiscalYear',
            'volatility_1month',
            'Forecast_FinancialStatement FiscalYear',
            'MA_gap_1month',
            'pbr',
            'Result_FinancialStatement FiscalYear',
            'return_1month',
            'ema_12',
            'Result_FinancialStatement TotalAssets',
            'signal',
            'Previous_FinancialStatement NetIncome',
            'per',
            'Result_FinancialStatement CashFlowsFromOperatingActivities',
            'Result_FinancialStatement CashFlowsFromInvestingActivities',
            'ema_10']

FEATURES_HIGH = ['MA_gap_2month_high',
                 'MA_gap_3month_high',
                 'volatility_2month_high',
                 'volatility_3month_high',
                 'Result_Dividend FiscalYear',
                 'return_3month_high',
                 'Forecast_Dividend FiscalYear',
                 'volatility_1month_high',
                 'Forecast_FinancialStatement FiscalYear',
                 'MA_gap_1month_high',
                 'pbr',
                 'Result_FinancialStatement FiscalYear',
                 'return_1month_high',
                 'ema_12',
                 'Result_FinancialStatement TotalAssets',
                 'signal',
                 'Previous_FinancialStatement NetIncome',
                 'per',
                 'Result_FinancialStatement CashFlowsFromOperatingActivities',
                 'Result_FinancialStatement CashFlowsFromInvestingActivities',
                 'ema_10']

FEATURES_LOW = ['MA_gap_2month_low',
                'MA_gap_3month_low',
                'volatility_2month_low',
                'volatility_3month_low',
                'Result_Dividend FiscalYear',
                'return_3month_low',
                'Forecast_Dividend FiscalYear',
                'volatility_1month_low',
                'Forecast_FinancialStatement FiscalYear',
                'MA_gap_1month_low',
                'pbr',
                'Result_FinancialStatement FiscalYear',
                'return_1month_low',
                'ema_12',
                'Result_FinancialStatement TotalAssets',
                'signal',
                'Previous_FinancialStatement NetIncome',
                'per',
                'Result_FinancialStatement CashFlowsFromOperatingActivities',
                'Result_FinancialStatement CashFlowsFromInvestingActivities',
                'ema_10']

In [10]:
section_products = {
    "First Section (Domestic)" : 1,
    "JASDAQ(Standard / Domestic)" :2,
    "Second Section(Domestic)" :3,
    "Mothers (Domestic)" : 4,
    "JASDAQ(Growth/Domestic)" :5
}

In [11]:
def calculate_glossary_of_financial_analysis(row):
    operating_profit_margin = 0
    ordinary_profit_margin = 0
    net_profit_margin = 0
    total_asset_turnover = 0
    net_sales_growth_rate = 0
    ordinary_income_growth_rate = 0
    operationg_income_growth_rate = 0
    total_assets_growth_rate = 0
    net_assets_growth_rate = 0
    eps = 0
    bps = 0
    roe = 0

    # 売上高営業利益率 売上高営業利益率（％）＝営業利益÷売上高×100
    if row['Result_FinancialStatement NetSales'] != 0:
        operating_profit_margin = \
            row['Result_FinancialStatement OperatingIncome'] / \
            row['Result_FinancialStatement NetSales'] * 100
    # 売上高経常利益率　売上高経常利益率（％）＝経常利益÷売上高×100
    if row['Result_FinancialStatement NetSales'] != 0:
        ordinary_profit_margin = \
            row['Result_FinancialStatement OrdinaryIncome'] / \
            row['Result_FinancialStatement NetSales'] * 100
    # 売上高純履歴率　売上高純利益率（％）＝当期純利益÷売上高×100
    if row['Result_FinancialStatement NetSales'] != 0:
        net_profit_margin = row['Result_FinancialStatement NetIncome'] / \
                            row['Result_FinancialStatement NetSales'] * 100
    # 総資本回転率 総資本回転率（％）＝売上高÷総資本（自己資本＋他人資本）×100
    if row['Result_FinancialStatement NetAssets'] != 0:
        total_asset_turnover = row['Result_FinancialStatement NetSales'] / \
                            row['Result_FinancialStatement NetAssets'] * 100
    # 売上高増加率
    if row['Previous_FinancialStatement NetSales'] != 0:
        net_sales_growth_rate = \
            (row['Result_FinancialStatement NetSales'] -
            row['Previous_FinancialStatement NetSales']) / \
            row['Previous_FinancialStatement NetSales'] * 100
    # 経常利益増加率
    if row['Previous_FinancialStatement OrdinaryIncome'] != 0:
        ordinary_income_growth_rate = \
            (row['Result_FinancialStatement OrdinaryIncome'] -
            row['Previous_FinancialStatement OrdinaryIncome']) / \
            row['Previous_FinancialStatement OrdinaryIncome'] * 100

    # 営業利益増加率
    if row['Previous_FinancialStatement OperatingIncome'] != 0:
        operationg_income_growth_rate = \
            (row['Result_FinancialStatement OperatingIncome'] -
            row['Previous_FinancialStatement OperatingIncome']) / \
            row['Previous_FinancialStatement OperatingIncome'] * 100
    # 総資本増加率
    if row['Previous_FinancialStatement TotalAssets'] != 0:
        total_assets_growth_rate = \
            (row['Result_FinancialStatement TotalAssets'] -
            row['Previous_FinancialStatement TotalAssets']) / \
            row['Previous_FinancialStatement TotalAssets'] * 100
    # 純資本増加率
    if row['Previous_FinancialStatement NetAssets'] != 0:
        net_assets_growth_rate = \
            (row['Result_FinancialStatement NetAssets'] -
            row['Previous_FinancialStatement NetAssets']) / \
            row['Previous_FinancialStatement NetAssets'] * 100
    # 一株当たり当期純利益（EPS）
    if row['IssuedShareEquityQuote IssuedShare'] != 0:
        eps = row['Result_FinancialStatement NetIncome'] / \
            row['IssuedShareEquityQuote IssuedShare']
        # BPS 一株当たり純資産（円） ＝ 純資産 ÷ 発行済株式総数
        bps = row['Result_FinancialStatement NetAssets'] / \
            row['IssuedShareEquityQuote IssuedShare']
        # ROE EPS（一株当たり利益）÷ BPS（一株当たり純資産）× 100
        if bps > 0:
            roe = eps / bps * 100
    return pd.Series(
        [operating_profit_margin, ordinary_profit_margin,
            net_profit_margin, total_asset_turnover,
            net_sales_growth_rate, ordinary_income_growth_rate,
            operationg_income_growth_rate, total_assets_growth_rate,
            net_assets_growth_rate, eps, bps, roe])

# 特徴量の生成

In [12]:
TRAIN_END = "2017-12-31"
VAL_START = "2018-02-01"
VAL_END = "2018-12-01"
TEST_START = "2019-01-01"

In [29]:
SELECT_FIN_DATA_COLUMNS = ['Result_FinancialStatement FiscalYear',
                           'Result_FinancialStatement NetSales',
                           'Result_FinancialStatement OperatingIncome',
                           'Result_FinancialStatement OrdinaryIncome',
                           'Result_FinancialStatement NetIncome',
                           'Result_FinancialStatement TotalAssets',
                           'Result_FinancialStatement NetAssets',
                           'Result_FinancialStatement '
                           'CashFlowsFromOperatingActivities',
                           'Result_FinancialStatement '
                           'CashFlowsFromFinancingActivities',
                           'Result_FinancialStatement '
                           'CashFlowsFromInvestingActivities',
                           'Forecast_FinancialStatement FiscalYear',
                           'Forecast_FinancialStatement NetSales',
                           'Forecast_FinancialStatement OperatingIncome',
                           'Forecast_FinancialStatement OrdinaryIncome',
                           'Forecast_FinancialStatement NetIncome',
                           'Result_Dividend FiscalYear',
                           'Result_Dividend QuarterlyDividendPerShare',
                           'Result_Dividend AnnualDividendPerShare',
                           'Forecast_Dividend FiscalYear',
                           'Forecast_Dividend QuarterlyDividendPerShare',
                           'Forecast_Dividend AnnualDividendPerShare',
                           'IssuedShareEquityQuote IssuedShare',
                           'Section/Products', '33 Sector(Code)',
                           '17 Sector(Code)']

SECTION_PRODUCTS = {
    "First Section (Domestic)": 1,
    "JASDAQ(Standard / Domestic)": 2,
    "Second Section(Domestic)": 3,
    "Mothers (Domestic)": 4,
    "JASDAQ(Growth/Domestic)": 5
}

FEATURES = ['MA_gap_2month',
            'MA_gap_3month',
            'volatility_2month',
            'volatility_3month',
            'Result_Dividend FiscalYear',
            'return_3month',
            'Forecast_Dividend FiscalYear',
            'volatility_1month',
            'Forecast_FinancialStatement FiscalYear',
            'MA_gap_1month',
            'pbr',
            'Result_FinancialStatement FiscalYear',
            'return_1month',
            'ema_12',
            'Result_FinancialStatement TotalAssets',
            'signal',
            'Previous_FinancialStatement NetIncome',
            'per',
            'Result_FinancialStatement CashFlowsFromOperatingActivities',
            'Result_FinancialStatement CashFlowsFromInvestingActivities',
            'ema_10']

FEATURES_HIGH = ['MA_gap_2month_high',
                 'MA_gap_3month_high',
                 'volatility_2month_high',
                 'volatility_3month_high',
                 'Result_Dividend FiscalYear',
                 'return_3month_high',
                 'Forecast_Dividend FiscalYear',
                 'volatility_1month_high',
                 'Forecast_FinancialStatement FiscalYear',
                 'MA_gap_1month_high',
                 'pbr',
                 'Result_FinancialStatement FiscalYear',
                 'return_1month_high',
                 'ema_12',
                 'Result_FinancialStatement TotalAssets',
                 'signal',
                 'Previous_FinancialStatement NetIncome',
                 'per',
                 'Result_FinancialStatement CashFlowsFromOperatingActivities',
                 'Result_FinancialStatement CashFlowsFromInvestingActivities',
                 'ema_10']

FEATURES_LOW = ['MA_gap_2month_low',
                'MA_gap_3month_low',
                'volatility_2month_low',
                'volatility_3month_low',
                'Result_Dividend FiscalYear',
                'return_3month_low',
                'Forecast_Dividend FiscalYear',
                'volatility_1month_low',
                'Forecast_FinancialStatement FiscalYear',
                'MA_gap_1month_low',
                'pbr',
                'Result_FinancialStatement FiscalYear',
                'return_1month_low',
                'ema_12',
                'Result_FinancialStatement TotalAssets',
                'signal',
                'Previous_FinancialStatement NetIncome',
                'per',
                'Result_FinancialStatement CashFlowsFromOperatingActivities',
                'Result_FinancialStatement CashFlowsFromInvestingActivities',
                'ema_10']


In [13]:
def get_features_for_predict(dfs, code):
    """
    Args:
        dfs (dict)  : dict of pd.DataFrame include stock_fin, stock_price
        code (int)  : A local code for a listed company
    Returns:
        feature DataFrame (pd.DataFrame)
    """
    # おおまかな手順の1つ目
    # stock_finデータを読み込み
    stock_fin = dfs["stock_fin"].copy()
    
    stock_list = dfs["stock_list"].copy()
    stock_fin = pd.merge(stock_fin, stock_list, on=["Local Code"] )

    # 特定の銘柄コードのデータに絞る
    fin_data = stock_fin[stock_fin["Local Code"] == code].copy()
    # 日付列をpd.Timestamp型に変換してindexに設定
    fin_data["datetime"] = pd.to_datetime(fin_data["base_date"])
    fin_data.set_index("datetime", inplace=True)
    # fin_dataを選択
    fin_data = fin_data[COLUMNS]
    fin_data = fin_data.join(fin_data[['Result_FinancialStatement NetSales', 'Result_FinancialStatement OperatingIncome', 
                                   'Result_FinancialStatement OrdinaryIncome', 'Result_FinancialStatement NetIncome', 
                                   'Result_FinancialStatement TotalAssets', 'Result_FinancialStatement NetAssets',
                                   'Result_FinancialStatement CashFlowsFromOperatingActivities', 
                                   'Result_FinancialStatement CashFlowsFromFinancingActivities',
                                   'Result_FinancialStatement CashFlowsFromInvestingActivities']].rename(columns =
                                                                                                         {'Result_FinancialStatement NetSales': 'Previous_FinancialStatement NetSales',
                                                                                                          'Result_FinancialStatement OperatingIncome': 'Previous_FinancialStatement OperatingIncome', 
                                                                                                          'Result_FinancialStatement OrdinaryIncome': 'Previous_FinancialStatement OrdinaryIncome', 
                                                                                                          'Result_FinancialStatement NetIncome':'Previous_FinancialStatement NetIncome', 
                                                                                                          'Result_FinancialStatement TotalAssets': 'Previous_FinancialStatement TotalAssets', 
                                                                                                          'Result_FinancialStatement NetAssets':'Previous_FinancialStatement NetAssets',
                                                                                                          'Result_FinancialStatement CashFlowsFromOperatingActivities': 'Previous_FinancialStatement CashFlowsFromOperatingActivities', 
                                                                                                          'Result_FinancialStatement CashFlowsFromFinancingActivities':'Previous_FinancialStatement CashFlowsFromFinancingActivities',
                                                                                                          'Result_FinancialStatement CashFlowsFromInvestingActivities':'Previous_FinancialStatement CashFlowsFromInvestingActivities'}).shift(-1))
    fin_data[['operating_profit_margin', 'ordinary_profit_margin', 'net_profit_margin', 'total_asset_turnover',
         'net_sales_growth_rate', 'ordinary_income_growth_rate', 'operationg_income_growth_rate',
          'total_assets_growth_rate', 'net_assets_growth_rate', 'eps', 'bps', 'roe']] = fin_data.apply(calculate_glossary_of_financial_analysis, axis=1)

    # 欠損値処理
    fin_feats = fin_data.fillna(0)

    # おおまかな手順の2つ目
    # stock_priceデータを読み込む
    price = dfs["stock_price"].copy()
    # 特定の銘柄コードのデータに絞る
    price_data = price[price["Local Code"] == code].copy()
    # 日付列をpd.Timestamp型に変換してindexに設定
    price_data["datetime"] = pd.to_datetime(price_data["EndOfDayQuote Date"])
    price_data.set_index("datetime", inplace=True)
    # 終値のみに絞る
    feats = price_data[["EndOfDayQuote ExchangeOfficialClose"]].copy()
    # 終値の20営業日リターン
    feats["return_1month"] = feats["EndOfDayQuote ExchangeOfficialClose"].pct_change(20)
    # 終値の40営業日リターン
    feats["return_2month"] = feats["EndOfDayQuote ExchangeOfficialClose"].pct_change(40)
    # 終値の60営業日リターン
    feats["return_3month"] = feats["EndOfDayQuote ExchangeOfficialClose"].pct_change(60)
    # 終値の20営業日ボラティリティ
    feats["volatility_1month"] = (
        np.log(feats["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(20).std()
    )
    # 終値の40営業日ボラティリティ
    feats["volatility_2month"] = (
        np.log(feats["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(40).std()
    )
    # 終値の60営業日ボラティリティ
    feats["volatility_3month"] = (
        np.log(feats["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(60).std()
    )
    # 終値と20営業日の単純移動平均線の乖離
    feats["MA_gap_1month"] = feats["EndOfDayQuote ExchangeOfficialClose"] / (
        feats["EndOfDayQuote ExchangeOfficialClose"].rolling(20).mean()
    )
    # 終値と40営業日の単純移動平均線の乖離
    feats["MA_gap_2month"] = feats["EndOfDayQuote ExchangeOfficialClose"] / (
        feats["EndOfDayQuote ExchangeOfficialClose"].rolling(40).mean()
    )
    # 終値と60営業日の単純移動平均線の乖離
    feats["MA_gap_3month"] = feats["EndOfDayQuote ExchangeOfficialClose"] / (
        feats["EndOfDayQuote ExchangeOfficialClose"].rolling(60).mean()
    )
    
    # EWMA
    ALPHA = 0.25
    feats["EWMA"] = feats["EndOfDayQuote ExchangeOfficialClose"]

    for t in zip(feats.index, feats.index[1:]):
        feats.loc[t[1], "EWMA"] = ALPHA * feats.loc[t[1], "EndOfDayQuote ExchangeOfficialClose"] + (1 - ALPHA) * feats.loc[t[0], "EWMA"]
    
    # EMA 10日
    feats["ema_10"] = feats["EndOfDayQuote ExchangeOfficialClose"].ewm(span=10).mean()
    
    # MACD 
    # EMA12
    feats["ema_12"] = feats["EndOfDayQuote ExchangeOfficialClose"].ewm(span=12).mean()
    # EMA 26
    feats["ema_26"] = feats["EndOfDayQuote ExchangeOfficialClose"].ewm(span=26).mean()
    feats["macd"] = feats["ema_12"] - feats["ema_26"]
    feats["signal"] = feats["macd"].ewm(span=9).mean()
    
    # PBR 株価 ÷ BPS（1株あたり純資産）
    feats["pbr"] = feats["EndOfDayQuote ExchangeOfficialClose"] / fin_data["bps"]
    # PER 株価 ÷ 1株当たり利益（EPS）
    feats["per"] = feats["EndOfDayQuote ExchangeOfficialClose"] / fin_data["eps"]

    # おおまかな手順の3つ目
    # 欠損値処理
    feats = feats.fillna(0)
    # 元データのカラムを削除
    feats = feats.drop(["EndOfDayQuote ExchangeOfficialClose"], axis=1)

    # 財務データの特徴量とマーケットデータの特徴量のインデックスを合わせる
    feats = feats.loc[feats.index.isin(fin_feats.index)]
    fin_feats = fin_feats.loc[fin_feats.index.isin(feats.index)]

    # データを結合
    feats = pd.concat([feats, fin_feats], axis=1).dropna()

    # 欠損値処理を行います。
    feats = feats.replace([np.inf, -np.inf], 0)
    
    # 市場・商品区分を数値に変換
    feats["Section/Products"] = section_products[feats["Section/Products"][0]]
    # 銘柄コードを設定
    feats["code"] = code

    return feats

In [14]:
def get_features_and_label(dfs, codes, feature, label):
    """
    Args:
        dfs (dict[pd.DataFrame]): loaded data
        codes  (array) : target codes
        feature (pd.DataFrame): features
        label (str) : label column name
    Returns:
        train_X (pd.DataFrame): training data
        train_y (pd.DataFrame): label for train_X
        val_X (pd.DataFrame): validation data
        val_y (pd.DataFrame): label for val_X
        test_X (pd.DataFrame): test data
        test_y (pd.DataFrame): label for test_X
    """
    # 分割データ用の変数を定義
    trains_X, vals_X, tests_X = [], [], []
    trains_y, vals_y, tests_y = [], [], []

    # 銘柄コード毎に特徴量を作成
    for code in tqdm(codes):
        # 特徴量取得
        feats = feature[feature["code"] == code]

        # stock_labelデータを読み込み
        stock_labels = dfs["stock_labels"].copy()
        # 特定の銘柄コードのデータに絞る
        stock_labels = stock_labels[stock_labels["Local Code"] == code]
        # 日付列をpd.Timestamp型に変換してindexに設定
        stock_labels["datetime"] = pd.to_datetime(stock_labels["base_date"])
        stock_labels.set_index("datetime", inplace=True)

        # 特定の目的変数に絞る
        labels = stock_labels[label]
        # nanを削除
        labels.dropna(inplace=True)

        if feats.shape[0] > 0 and labels.shape[0] > 0:
            # 特徴量と目的変数のインデックスを合わせる
            labels = labels.loc[labels.index.isin(feats.index)]
            feats = feats.loc[feats.index.isin(labels.index)]
            labels.index = feats.index

            # データを分割（ホールドアウト法）
            _train_X = feats[: TRAIN_END].copy()
            _val_X = feats[VAL_START : VAL_END].copy()
            _test_X = feats[TEST_START :].copy()

            _train_y = labels[: TRAIN_END].copy()
            _val_y = labels[VAL_START : VAL_END].copy()
            _test_y = labels[TEST_START :].copy()

            # データを配列に格納 (後ほど結合するため)
            trains_X.append(_train_X)
            vals_X.append(_val_X)
            tests_X.append(_test_X)

            trains_y.append(_train_y)
            vals_y.append(_val_y)
            tests_y.append(_test_y)

    # 銘柄毎に作成した説明変数データを結合します。
    train_X = pd.concat(trains_X)
    val_X = pd.concat(vals_X)
    test_X = pd.concat(tests_X)
    # 銘柄毎に作成した目的変数データを結合します。
    train_y = pd.concat(trains_y)
    val_y = pd.concat(vals_y)
    test_y = pd.concat(tests_y)

    return train_X, train_y, val_X, val_y, test_X, test_y

In [15]:
def get_codes(dfs):
    """
    Args:
        dfs (dict[pd.DataFrame]): loaded data
    Returns:
        array: list of stock codes
    """
    stock_list = dfs["stock_list"].copy()
    # 予測対象の銘柄コードを取得
    codes = stock_list[stock_list["prediction_target"] == True][
        "Local Code"
    ].values
    return codes

In [9]:
from models import ModelETR as cls_etr
from models import ModelLGB as cls_lgb
from models import ModelXGB as cls_xgb
from models import ModelCGB as cls_cgb
from models import ModelNN as cls_nn

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [10]:
from predictor import ScoringService as cls

In [44]:
model_path = os.path.join(os.path.dirname("__file__"), "../model")
test_X_path = os.path.join(os.path.dirname("__file__"), "../../high_low_datas/test_X")
test_y_path = os.path.join(os.path.dirname("__file__"), "../../high_low_datas/test_y")
val_X_path = os.path.join(os.path.dirname("__file__"), "../../high_low_datas/val_X")
val_y_path = os.path.join(os.path.dirname("__file__"), "../../high_low_datas/val_y")
train_X_path = os.path.join(os.path.dirname("__file__"), "../../high_low_datas/train_X")
train_y_path = os.path.join(os.path.dirname("__file__"), "../../high_low_datas/train_y")

In [45]:
# 対象の目的変数を定義
labels = {
#    "label_high_5",
#    "label_high_10",
    "label_high_20",
#    "label_low_5",
#    "label_low_10",
    "label_low_20",
}

In [46]:
# 特徴量追加済みデータ
proceed_datas = {
    "train_X",
    "train_y",
    "val_X",
    "val_y",
    "test_X",
    "test_y"
}

In [47]:
train_X = {}
train_y = {}
val_X = {}
val_y = {}

In [48]:
for label in labels:
    data_X = os.path.join(train_X_path, f"train_X_{label}.pkl")
    with open(data_X , "rb") as f:
        train_X[label] = pickle.load(f)
    data_y = os.path.join(train_y_path, f"train_y_{label}.pkl")
    with open(data_y , "rb") as f:
        train_y[label] = pickle.load(f)

In [49]:
for label in labels:
    data_X = os.path.join(val_X_path, f"val_X_{label}.pkl")
    with open(data_X , "rb") as f:
        val_X[label] = pickle.load(f)
    data_y = os.path.join(val_y_path, f"val_y_{label}.pkl")
    with open(data_y , "rb") as f:
        val_y[label] = pickle.load(f)

In [50]:
def get_feature_columns(dfs, train_X, column_group="fundamental+technical"):
    # 特徴量グループを定義
    # ファンダメンタル
    fundamental_cols = dfs["stock_fin"].select_dtypes("float64").columns
    fundamental_cols = fundamental_cols[
        fundamental_cols != "Result_Dividend DividendPayableDate"
        ]
    fundamental_cols = fundamental_cols[fundamental_cols != "Local Code"]
    # 価格変化率
    returns_cols = [x for x in train_X.columns if "return" in x]
    # テクニカル
    technical_cols = [
        x for x in train_X.columns if
        (x not in fundamental_cols) and (x != "code")
    ]
    columns = {
        #"fundamental_only": fundamental_cols,
        #"return_only": returns_cols,
        #"technical_only": technical_cols,
        #"fundamental+technical": list(fundamental_cols) + list(
        #    technical_cols),
        #"selected_high_columns":FEATURES_HIGH,
        "selected_low_columns":FEATURES_LOW,
        "selected_columns":FEATURES,
    }
    return columns[column_group]

In [51]:
fundamental_cols = dfs["stock_fin"].select_dtypes("float64").columns
fundamental_cols = fundamental_cols[
    fundamental_cols != "Result_Dividend DividendPayableDate"
    ]
fundamental_cols = fundamental_cols[fundamental_cols != "Local Code"]
# 価格変化率
returns_cols = [x for x in train_X['label_high_20'].columns if "return" in x]
# テクニカル
technical_cols = [
    x for x in train_X['label_high_20'].columns if
    (x not in fundamental_cols) and (x != "code")]

In [52]:
# モデルを定義
models = {
    "extraTree": cls_etr,
    "lgb": cls_lgb,
    "xgb": cls_xgb,
    "cgb": cls_cgb,
    "nn": cls_nn
}

# 学習用データセット定義
columns = {
    #"fundamental_only": fundamental_cols,
    #"return_only": returns_cols,
    #"technical_only": technical_cols,
    #"fundamental+technical": list(fundamental_cols) + list(technical_cols),
    #"selected_columns":FEATURES,
    "selected_high_columns":FEATURES_HIGH,
    "selected_low_columns":FEATURES_LOW,
}

In [53]:
labels

{'label_high_20', 'label_low_20'}

In [54]:
test_X = {}
test_y = {}

In [55]:
for label in labels:
    data_X = os.path.join(test_X_path, f"test_X_{label}.pkl")
    with open(data_X , "rb") as f:
        test_X[label] = pickle.load(f)
    data_y = os.path.join(test_y_path, f"test_y_{label}.pkl")
    with open(data_y , "rb") as f:
        test_y[label] = pickle.load(f)

In [56]:
test_X[label]

,EndOfDayQuote High,EndOfDayQuote Low,return_1month,return_2month,return_3month,volatility_1month,volatility_2month,volatility_3month,MA_gap_1month,MA_gap_2month,MA_gap_3month,return_1month_high,return_2month_high,return_3month_high,volatility_1month_high,volatility_2month_high,volatility_3month_high,MA_gap_1month_high,MA_gap_2month_high,MA_gap_3month_high,return_1month_low,return_2month_low,return_3month_low,volatility_1month_low,volatility_2month_low,volatility_3month_low,MA_gap_1month_low,MA_gap_2month_low,MA_gap_3month_low,EWMA,ema_10,ema_12,ema_26,macd,signal,pbr,per,Result_FinancialStatement FiscalYear,Result_FinancialStatement NetSales,Result_FinancialStatement OperatingIncome,Result_FinancialStatement OrdinaryIncome,Result_FinancialStatement NetIncome,Result_FinancialStatement TotalAssets,Result_FinancialStatement NetAssets,Result_FinancialStatement CashFlowsFromOperatingActivities,Result_FinancialStatement CashFlowsFromFinancingActivities,Result_FinancialStatement CashFlowsFromInvestingActivities,Forecast_FinancialStatement FiscalYear,Forecast_FinancialStatement NetSales,Forecast_FinancialStatement OperatingIncome,Forecast_FinancialStatement OrdinaryIncome,Forecast_FinancialStatement NetIncome,Result_Dividend FiscalYear,Result_Dividend QuarterlyDividendPerShare,Result_Dividend AnnualDividendPerShare,Forecast_Dividend FiscalYear,Forecast_Dividend QuarterlyDividendPerShare,Forecast_Dividend AnnualDividendPerShare,IssuedShareEquityQuote IssuedShare,Section/Products,33 Sector(Code),17 Sector(Code),Previous_FinancialStatement NetSales,Previous_FinancialStatement OperatingIncome,Previous_FinancialStatement OrdinaryIncome,Previous_FinancialStatement NetIncome,Previous_FinancialStatement TotalAssets,Previous_FinancialStatement NetAssets,Previous_FinancialStatement CashFlowsFromOperatingActivities,Previous_FinancialStatement CashFlowsFromFinancingActivities,Previous_FinancialStatement CashFlowsFromInvestingActivities,operating_profit_margin,ordinary_profit_margin,net_profit_margin,total_asset_turnover,net_sales_growth_rate,ordinary_income_growth_rate,operationg_income_growth_rate,total_assets_growth_rate,net_assets_growth_rate,eps,bps,roe,code
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-02-08,2856.0,2788.0,-0.029990,-0.115094,-0.068212,0.009129,0.019928,0.019123,0.999325,0.979481,0.952214,-0.026253,-0.107500,-0.072727,0.009628,0.019694,0.018517,0.991072,0.968525,0.942856,-0.030261,-0.120505,-0.073754,0.007849,0.019214,0.017127,1.004767,0.986875,0.960781,2809.628787,2808.901841,2810.251424,2839.679516,-29.428092,-39.914841,9.752383e+05,1.229596e+07,2019.0,197783.0,3280.0,3923.0,2501.0,130246.0,31533.0,0.0,0.0,0.0,2019.0,267000.0,4800.0,4600.0,3200.0,2018.0,60.0,60.0,2019.0,60.0,60.0,10928283.0,1,50,1,256151.0,3831.0,4434.0,2914.0,114673.0,31996.0,-4118.0,5958.0,-2494.0,1.658383,1.983487,1.264517,627.225446,-22.786559,-11.524583,-14.382668,13.580355,-1.447056,2.288557e-04,0.002885,7.931373,1301
2019-05-13,2826.0,2656.0,0.005413,-0.071333,-0.019359,0.013405,0.015306,0.013476,1.001132,0.974970,0.963442,0.013266,-0.062687,-0.012579,0.011730,0.012384,0.010971,0.993421,0.966095,0.955866,-0.037332,-0.114076,-0.059823,0.009037,0.012636,0.011340,1.010995,0.983219,0.970946,2754.939314,2761.369792,2766.126352,2801.777661,-35.651308,-37.034735,9.515626e+05,1.044825e+07,2019.0,256151.0,3831.0,4434.0,2914.0,114673.0,31996.0,-4118.0,5958.0,-2494.0,2020.0,286000.0,5500.0,5400.0,3800.0,2019.0,70.0,70.0,2020.0,70.0,70.0,10928283.0,1,50,1,62644.0,770.0,1015.0,706.0,122181.0,31540.0,0.0,0.0,0.0,1.495602,1.731010,1.137610,800.571946,308.899496,336.847291,397.532468,-6.144982,1.445783,2.666476e-04,0.002928,9.107388,1301
2019-08-02,3045.0,2985.0,-0.004950,0.136878,0.107641,0.009575,0.011843,0.012424,0.987424,1.030734,1.049274,0.004950,0.126109,0.098485,0.006291,0.010259,0.011515,0.981685,1.024830,1.041925,-0.008306,0.125566,0.099043,0.007971,0.010273,0.012261,0.995707,1.041307,1.060642,3052.050178

In [57]:
# 結果保存用
all_results = dict()
# モデル毎に処理
for model in tqdm(models.keys()):
    print(model)
    all_results[model] = dict()
    # データセット毎に処理
    for col in columns.keys():
        print(col)
        result = dict()
        # 目的変数毎に処理
        for label in tqdm(labels):
            print(label)
            if len(test_X[label][columns[col]]) > 0:
                result[label] = test_X[label][["code"]].copy()
                result[label]["datetime"] = test_X[label][columns[col]].index
                if model == "nn":
                    models[model].create_model(train_X[label][columns[col]].values, train_y[label], val_X[label][columns[col]].values, val_y[label], label)
                else:
                    models[model].create_model(train_X[label][columns[col]], train_y[label], label)
                # 予測
                result[label]["predict"] = models[model].predict(label, test_X[label][columns[col]].values)
                result[label]["predict_dir"] = np.sign(result[label]["predict"])
                # 実際の結果
                result[label]["actual"] = test_y[label].values
                result[label]["actual_dir"] = np.sign(result[label]["actual"])
                result[label].dropna(inplace=True)
                
        all_results[model][col] = result

extraTree
selected_high_columns


label_high_20
model et predict
label_low_20
model et predict
selected_low_columns


label_high_20
model et predict
label_low_20
model et predict
lgb
selected_high_columns


label_high_20
lgb model predict
label_low_20
lgb model predict
selected_low_columns


label_high_20
lgb model predict
label_low_20
lgb model predict
xgb
selected_high_columns


...

73:	learn: 0.0665789	total: 250ms	remaining: 501ms
74:	learn: 0.0665250	total: 255ms	remaining: 500ms
75:	learn: 0.0664792	total: 259ms	remaining: 498ms
76:	learn: 0.0664159	total: 263ms	remaining: 496ms
77:	learn: 0.0663669	total: 267ms	remaining: 493ms
78:	learn: 0.0663253	total: 270ms	remaining: 489ms
79:	learn: 0.0662947	total: 273ms	remaining: 485ms
80:	learn: 0.0662507	total: 276ms	remaining: 481ms
81:	learn: 0.0661826	total: 279ms	remaining: 477ms
82:	learn: 0.0660796	total: 283ms	remaining: 474ms
83:	learn: 0.0660501	total: 286ms	remaining: 470ms
84:	learn: 0.0660069	total: 289ms	remaining: 466ms
85:	learn: 0.0659657	total: 292ms	remaining: 462ms
86:	learn: 0.0658661	total: 296ms	remaining: 459ms
87:	learn: 0.0658034	total: 299ms	remaining: 455ms
88:	learn: 0.0657763	total: 302ms	remaining: 452ms
89:	learn: 0.0657340	total: 305ms	remaining: 448ms
90:	learn: 0.0656938	total: 308ms	remaining: 444ms
91:	learn: 0.0655978	total: 311ms	remaining: 440ms
92:	learn: 0.0655373	total: 314

label_high_20
0:	learn: 0.1796897	total: 4.62ms	remaining: 458ms
1:	learn: 0.1792103	total: 9.26ms	remaining: 454ms
2:	learn: 0.1788772	total: 12.7ms	remaining: 409ms
3:	learn: 0.1785406	total: 15.6ms	remaining: 374ms
4:	learn: 0.1782070	total: 18.9ms	remaining: 359ms
5:	learn: 0.1773057	total: 22.2ms	remaining: 348ms
6:	learn: 0.1767623	total: 25.3ms	remaining: 336ms
7:	learn: 0.1766404	total: 28.4ms	remaining: 326ms
8:	learn: 0.1762570	total: 31.9ms	remaining: 323ms
9:	learn: 0.1756809	total: 34.8ms	remaining: 313ms
10:	learn: 0.1753592	total: 37.9ms	remaining: 307ms
11:	learn: 0.1752469	total: 41.2ms	remaining: 302ms
12:	learn: 0.1748964	total: 44.5ms	remaining: 298ms
13:	learn: 0.1745812	total: 47.6ms	remaining: 292ms
14:	learn: 0.1745062	total: 51ms	remaining: 289ms
15:	learn: 0.1743339	total: 54.2ms	remaining: 285ms
16:	learn: 0.1739391	total: 57ms	remaining: 278ms
17:	learn: 0.1736658	total: 61.5ms	remaining: 280ms
18:	learn: 0.1736053	total: 65.1ms	remaining: 278ms
19:	learn: 0

80:	learn: 0.0660153	total: 273ms	remaining: 476ms
81:	learn: 0.0659599	total: 277ms	remaining: 473ms
82:	learn: 0.0658939	total: 281ms	remaining: 471ms
83:	learn: 0.0658409	total: 285ms	remaining: 468ms
84:	learn: 0.0657484	total: 288ms	remaining: 464ms
85:	learn: 0.0657039	total: 291ms	remaining: 461ms
86:	learn: 0.0656686	total: 295ms	remaining: 457ms
87:	learn: 0.0656183	total: 298ms	remaining: 454ms
88:	learn: 0.0655747	total: 301ms	remaining: 450ms
89:	learn: 0.0655436	total: 304ms	remaining: 446ms
90:	learn: 0.0655161	total: 307ms	remaining: 442ms
91:	learn: 0.0654533	total: 310ms	remaining: 439ms
92:	learn: 0.0654257	total: 314ms	remaining: 435ms
93:	learn: 0.0653679	total: 317ms	remaining: 431ms
94:	learn: 0.0653219	total: 320ms	remaining: 428ms
95:	learn: 0.0652874	total: 323ms	remaining: 424ms
96:	learn: 0.0652541	total: 327ms	remaining: 421ms
97:	learn: 0.0652057	total: 330ms	remaining: 418ms
98:	learn: 0.0651166	total: 333ms	remaining: 414ms
99:	learn: 0.0650851	total: 337

label_high_20
Epoch 1/80
993/993 [==============================] - 22s 17ms/step - loss: 0.3179 - mse: 0.3179 - val_loss: 0.0211 - val_mse: 0.0211
Epoch 2/80
993/993 [==============================] - 15s 15ms/step - loss: 0.0504 - mse: 0.0504 - val_loss: 0.0201 - val_mse: 0.0201
Epoch 3/80
993/993 [==============================] - 15s 15ms/step - loss: 0.0414 - mse: 0.0414 - val_loss: 0.0198 - val_mse: 0.0198
Epoch 4/80
993/993 [==============================] - 15s 15ms/step - loss: 0.0314 - mse: 0.0314 - val_loss: 0.0199 - val_mse: 0.0199
Epoch 5/80
993/993 [==============================] - 15s 15ms/step - loss: 0.0287 - mse: 0.0287 - val_loss: 0.0217 - val_mse: 0.0217
Epoch 6/80
993/993 [==============================] - 15s 15ms/step - loss: 0.0329 - mse: 0.0329 - val_loss: 0.0196 - val_mse: 0.0196
Epoch 7/80
993/993 [==============================] - 15s 15ms/step - loss: 0.0295 - mse: 0.0295 - val_loss: 0.0194 - val_mse: 0.0194
Epoch 8/80
993/993 [============================

label_high_20
Epoch 1/80
993/993 [==============================] - 21s 17ms/step - loss: 0.3043 - mse: 0.3043 - val_loss: 0.0206 - val_mse: 0.0206
Epoch 2/80
993/993 [==============================] - 15s 15ms/step - loss: 0.0484 - mse: 0.0484 - val_loss: 0.0202 - val_mse: 0.0202
Epoch 3/80
993/993 [==============================] - 16s 16ms/step - loss: 0.0404 - mse: 0.0404 - val_loss: 0.0198 - val_mse: 0.0198
Epoch 4/80
993/993 [==============================] - 16s 16ms/step - loss: 0.0313 - mse: 0.0313 - val_loss: 0.0193 - val_mse: 0.0193
Epoch 5/80
993/993 [==============================] - 15s 15ms/step - loss: 0.0286 - mse: 0.0286 - val_loss: 0.0204 - val_mse: 0.0204
Epoch 6/80
993/993 [==============================] - 15s 15ms/step - loss: 0.0332 - mse: 0.0332 - val_loss: 0.0194 - val_mse: 0.0194
Epoch 7/80
993/993 [==============================] - 16s 16ms/step - loss: 0.0298 - mse: 0.0298 - val_loss: 0.0193 - val_mse: 0.0193
Epoch 8/80
993/993 [============================

In [58]:
results = []
for model in all_results.keys():
    for col in all_results[model]:
        tmp = pd.concat(all_results[model][col])
        tmp["model"] = model
        tmp["feature"] = col
        results.append(tmp)
results = pd.concat(results)
results["label"] = [x[0] for x in results.index]
results.head(5)

code   datetime   predict  predict_dir   actual  actual_dir      model  \
              datetime                                                                             
label_high_20 2019-02-08  1301 2019-02-08  0.088172          1.0  0.07854         1.0  extraTree   
              2019-05-13  1301 2019-05-13  0.076896          1.0  0.04379         1.0  extraTree   
              2019-08-02  1301 2019-08-02  0.073419          1.0  0.00498         1.0  extraTree   
              2019-11-05  1301 2019-11-05  0.073235          1.0  0.00841         1.0  extraTree   
              2020-02-07  1301 2020-02-07  0.071771          1.0  0.01212         1.0  extraTree   

                                        feature          label  
              datetime                                          
label_high_20 2019-02-08  selected_high_columns  label_high_20  
              2019-05-13  selected_high_columns  label_high_20  
              2019-08-02  selected_high_columns  label_high_20  
              2019-11-05  selected_high_columns  label_high_20  
              2020-02-07  selected_high_columns  label_high_20

In [39]:
results[(results["model"] == "cgb") & (results["label"] == "label_low_20") & (results["feature"] == "selected_columns")]

code   datetime   predict  predict_dir   actual  actual_dir model           feature  \
             datetime                                                                                          
label_low_20 2019-02-08  1301 2019-02-08 -0.061614         -1.0  0.00604         1.0   cgb  selected_columns   
             2019-05-13  1301 2019-05-13 -0.053668         -1.0 -0.06317        -1.0   cgb  selected_columns   
             2019-08-02  1301 2019-08-02 -0.053491         -1.0 -0.08723        -1.0   cgb  selected_columns   
             2019-11-05  1301 2019-11-05 -0.055237         -1.0 -0.03800        -1.0   cgb  selected_columns   
             2020-02-07  1301 2020-02-07 -0.018230         -1.0 -0.17832        -1.0   cgb  selected_columns   
             2020-05-12  1301 2020-05-12 -0.082078         -1.0  0.00574         1.0   cgb  selected_columns   
             2020-08-07  1301 2020-08-07 -0.046376         -1.0 -0.02542        -1.0   cgb  selected_columns   
             2020-11-06  1301 2020-11-06 -0.043195         -1.0 -0.00899        -1.0   cgb  selected_columns   
             2019-02-05  1332 2019-02-05 -0.064794         -1.0 -0.02078        -1.0   cgb  selected_columns   
             2019-05-14  1332 2019-05-14 -0.045619         -1.0 -0.08223        -1.0   cgb  selected_columns   
             2019-08-05  1332 2019-08-05 -0.054838         -1.0 -0.06742        -1.0   cgb  selected_columns   
             2019-11-05  1332 2019-11-05 -0.058434         -1.0 -0.04560        -1.0   cgb  selected_columns   
             2020-02-05  1332 2020-02-05 -0.064170         -1.0 -0.21359        -1.0   cgb  selected_columns   
             2020-05-20  1332 2020-05-20 -0.094011         -1.0 -0.04124        -1.0   cgb  selected_columns   
             2020-08-03  1332 2020-08-03 -0.060620         -1.0 -0.01935        -1.0   cgb  selected_columns   
             2020-11-05  1332 2020-11-05 -0.052894         -1.0 -0.03837        -1.0   cgb  selected_columns   
             2019-02-04  1333 2019-02-04 -0.062379         -1.0 -0.05749        -1.0   cgb  selected_columns   
             2019-05-13  1333 2019-05-13 -0.059433         -1.0 -0.07331        -1.0   cgb  selected_columns   
             2019-08-05  1333 2019-08-05 -0.048522         -1.0 -0.13731        -1.0   cgb  selected_columns   
             2019-11-05  1333 2019-11-05 -0.060422         -1.0  0.00000         0.0   cgb  selected_columns   
             2020-02-03  1333 2020-02-03 -0.049289         -1.0 -0.16465        -1.0   cgb  selected_columns   
             2020-05-14  1333 2020-05-14 -0.094153         -1.0 -0.06464        -1.0   cgb  selected_columns   
             2020-08-03  1333 2020-08-03 -0.064773         -1.0  0.07270         1.0   cgb  selected_columns   
             2020-11-02  1333 2020-11-02 -0.058736         -1.0 -0.06435        -1.0   cgb  selected_columns   
             2019-01-31  1352 2019-01-31 -0.074980         -1.0 -0.04452        -1.0   cgb  selected_columns   
             2019-05-14  1352 2019-05-14 -0.053314         -1.0 -0.02436        -1.0   cgb  selected_columns   
             2019-07-30  1352 2019-07-30 -0.034985         -1.0 -0.07394        -1.0   cgb  selected_columns   
             2019-10-31  1352 2019-10-31 -0.048187         -1.0 -0.01174        -1.0   cgb  selected_columns   
             2020-01-31  1352 2020-01-31 -0.038222         -1.0 -0.16927        -1.0   cgb  selected_columns   
             2020-05-15  1352 2020-05-15 -0.097411         -1.0 -0.05006        -1.0   cgb  selected_columns   
             2020-07-30  1352 2020-07-30 -0.063962         -1.0 -0.07598        -1.0   cgb  selected_columns   
             2020-10-30  1352 2020-10-30 -0.035916         -1.0 -0.03247        -1.0   cgb  selected_columns   
             2019-01-07  1376 2019-01-07 -0.052185         -1.0 -0.08819        -1.0   cgb  selected_columns   
             2019-04-04  1376 2019-04-04 -0.060110         -1.0 -0.09248        -1.0   cgb  selected_colu

In [59]:
# 結果保存用変数
all_metrics = []

# データセット毎に処理
for feature in columns:
    print(feature)
    matrix = dict()
    # モデル毎に処理
    for model in models:
        print(model)
        # 目的変数毎に処理
        for label in labels:
            print(label)
            # 処理対象データに絞り込み
            tmp_df = results[(results["model"] == model) & (results["label"] == label) & (results["feature"] == feature)]
            if len(tmp_df) > 0:
                # RMSE
                rmse = np.sqrt(mean_squared_error(tmp_df["predict"], tmp_df["actual"]))
                # 精度
                accuracy = accuracy_score(tmp_df["predict_dir"], tmp_df["actual_dir"])
                # 相関係数
                corr = np.corrcoef(tmp_df["actual"], tmp_df["predict"])[0, 1]
                # 順位相関
                spearman_corr = spearmanr(tmp_df["actual"], tmp_df["predict"])[0]
                # 結果を保存
                matrix[label] = [rmse, accuracy, spearman_corr,corr, corr**2, feature, model, tmp_df.shape[0]]
        res = pd.DataFrame.from_dict(matrix).T
        res.columns = ["RMSE","accuracy","spearman_corr","corr","R^2 score","feature", "model", "# of samples"]
        all_metrics.append(res)
all_metrics = pd.concat(all_metrics)
all_metrics.reset_index()

selected_high_columns
extraTree
label_high_20
label_low_20
lgb
label_high_20
label_low_20
xgb
label_high_20
label_low_20
cgb
label_high_20
label_low_20
nn
label_high_20
label_low_20
selected_low_columns
extraTree
label_high_20
label_low_20
lgb
label_high_20
label_low_20
xgb
label_high_20
label_low_20
cgb
label_high_20
label_low_20
nn
label_high_20
label_low_20


,index,RMSE,accuracy,spearman_corr,corr,R^2 score,feature,model,# of samples
0,label_high_20,0.163115,0.832293,0.291721,0.270734,0.0732971,selected_high_columns,extraTree,32515
1,label_low_20,0.104899,0.867938,0.147146,0.128157,0.0164243,selected_high_columns,extraTree,32515
2,label_high_20,0.161497,0.832293,0.231564,0.265014,0.0702324,selected_high_columns,lgb,32515
3,label_low_20,0.103852,0.867723,0.159847,0.151335,0.0229023,selected_high_columns,lgb,32515
4,label_high_20,0.160455,0.832293,0.25497,0.281064,0.0789972,selected_high_columns,xgb,32515
5,label_low_20,0.10321,0.867938,0.183617,0.183633,0.0337212,selected_high_columns,xgb,32515
6,label_high_20,0.162384,0.830909,0.223258,0.250707,0.0628538,selected_high_columns,cgb,32515
7,label_low_20,0.103487,0.86763,0.163602,0.164026,0.0269044,selected_high_columns,cgb,32515
8,label_high_20,0.161211,0.832293,0.233462,0.265247,0.0703562,selected_high_columns,nn,32515
9,label_low_20,0.106013,0.867815,0.128035,0.138565,0.0192004,selected_high_columns,nn,32515


In [61]:
all_metrics[all_metrics.index == "label_low_20"].sort_values('model')

,RMSE,accuracy,spearman_corr,corr,R^2 score,feature,model,# of samples
label_low_20,0.103487,0.86763,0.163602,0.164026,0.0269044,selected_high_columns,cgb,32515
label_low_20,0.103742,0.867753,0.163248,0.156963,0.0246374,selected_low_columns,cgb,32515
label_low_20,0.104899,0.867938,0.147146,0.128157,0.0164243,selected_high_columns,extraTree,32515
label_low_20,0.104987,0.867938,0.139862,0.120235,0.0144564,selected_low_columns,extraTree,32515
label_low_20,0.103852,0.867723,0.159847,0.151335,0.0229023,selected_high_columns,lgb,32515
label_low_20,0.103464,0.867907,0.174184,0.161674,0.0261386,selected_low_columns,lgb,32515
label_low_20,0.106013,0.867815,0.128035,0.138565,0.0192004,selected_high_columns,nn,32515
label_low_20,0.108232,0.867876,0.125386,0.111038,0.0123294,selected_low_columns,nn,32515
label_low_20,0.10321,0.867938,0.183617,0.183633,0.0337212,selected_high_columns,xgb,32515
label_low_20,0.103286,0.867938,0.175408,0.167675,0.028115,selected_low_columns,xgb,32515
